daily_run estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, 70: 2185-2226. doi:10.1111/jofi.12269- 

Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [ ]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
from spx_data_update import UpdateSP500Data
import numpy as np
import pandas as pd
import nest_asyncio
from time import time
import plistlib
file_name = UpdateSP500Data.DATA_BASE_PATH / 'config.plist'
f = open(str(file_name), 'rb')
pl = plistlib.load(f)
illiquid_dict = pl['illiquid_equity']
illiquid_equity = sum(illiquid_dict.values()) * 0.5
illiquid_equity

536500.0

In [2]:
before = time()
#nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [3]:
z_score = np.array([-0.5, -1, -1.5])
leverage = np.array(range(1, 3, 1))
num_expiries = 3

after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 23.5
elapsed: 40.1
2615.04
21.71
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2737224.0729', currency='BASE', modelCode='')]


In [4]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [5]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-0.5,-1.0,-1.5
2018.12.21,2575,2530,2500
2019.01.18,2540,2445,2390
2019.02.15,2500,2365,2285


,-0.5,-1.0,-1.5
2018.12.21,15.20,6.95,4.2
2019.01.18,35.65,17.60,11.7
2019.02.15,45.85,22.20,14.4


,-0.5,-1.0,-1.5
2018.12.21,14.9/15.5,6.8/7.1,3.9/4.2
2019.01.18,35.3/36.0,17.4/17.8,11.5/11.9
2019.02.15,45.5/46.2,21.9/22.5,14.2/14.6


In [6]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-0.5,-1.0,-1.5
2018.12.21,-1.53%,-3.25%,-4.40%
2019.01.18,-2.87%,-6.50%,-8.61%
2019.02.15,-4.40%,-9.56%,-12.62%


In [7]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-0.5,-1.0,-1.5
2018.12.21,"$19,760","$9,035","$5,460"
2019.01.18,"$46,345","$24,640","$16,380"
2019.02.15,"$59,605","$31,080","$20,160"


,-0.5,-1.0,-1.5
2018.12.21,"$39,520","$18,070","$10,920"
2019.01.18,"$92,690","$47,520","$32,760"
2019.02.15,"$123,795","$62,160","$41,760"


In [8]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-0.5,-1.0,-1.5
2018.12.21,0.58%,0.27%,0.16%
2019.01.18,1.36%,0.67%,0.45%
2019.02.15,1.75%,0.85%,0.55%


,-0.5,-1.0,-1.5
2018.12.21,1.16%,0.53%,0.32%
2019.01.18,2.73%,1.35%,0.89%
2019.02.15,3.51%,1.70%,1.10%


In [9]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-0.5,-1.0,-1.5
Lev 1,,,
2018.12.21,13,13,13
2019.01.18,13,14,14
2019.02.15,13,14,14


,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,26,26,26
2019.01.18,26,27,28
2019.02.15,27,28,29


In [10]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-0.5,-1.0,-1.5
2018.12.21,"$647,618","$578,393","$535,818"
2019.01.18,"$628,703","$518,795","$433,535"
2019.02.15,"$589,963","$413,235","$340,060"


,-0.5,-1.0,-1.5
2018.12.21,"$1,295,237","$1,156,787","$1,071,637"
2019.01.18,"$1,257,407","$1,000,534","$867,070"
2019.02.15,"$1,225,309","$826,470","$704,410"


In [11]:
marge = trd_choice.margin(trd_choice.spot)
lots = trd_choice.option_lots([2], capital_at_risk)
margin = lots[0] * marge
display(margin.style.format('${:,.0f}'))

,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,"$1,295,237","$1,156,787","$1,071,637"
2019.01.18,"$1,257,407","$1,000,534","$867,070"
2019.02.15,"$1,225,309","$826,470","$704,410"


In [12]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-0.5,-1.0,-1.5
Lev 1,,,
2018.12.21,"$3,347,500","$3,289,000","$3,250,000"
2019.01.18,"$3,302,000","$3,423,000","$3,346,000"
2019.02.15,"$3,250,000","$3,311,000","$3,199,000"


,-0.5,-1.0,-1.5
Lev 2,,,
2018.12.21,"$6,695,000","$6,578,000","$6,500,000"
2019.01.18,"$6,604,000","$6,601,500","$6,692,000"
2019.02.15,"$6,750,000","$6,622,000","$6,626,500"


In [13]:
capital_at_risk

3273724.0729

In [14]:
illiquid_equity

536500.0